In [36]:
import cat4py as cat
import numpy as np

In [37]:
class Array(cat.Container):
    def __init__(self, dtype, pshape=None, filename=None, **kargs):
        self.dtype = np.dtype(dtype)
        self.kargs = kargs
        super(cat.Container, self).__init__(pshape, filename, **kargs)
    
    def iter_write(self):
        return cat.Container.iter_write(self, self.dtype)
    
    def iter_read(self, blockshape):
        return cat.Container.iter_read(self, blockshape, self.dtype)
    def __array__(self):
        return cat.Container.to_numpy(self, self.dtype)

def empty_array(dtype, shape, pshape=None, filename=None, **kargs):
    arr = Array(dtype, pshape, filename, **kargs)
    arr.updateshape(shape)
    return arr

In [38]:
a = empty_array(np.int32, shape=(10, 10))
print(a.psize)

100


In [39]:
for block, info in a.iter_write():
    block[:] = np.ones((info.shape), dtype=np.int32)

In [41]:
np.sum(a)

100